In [45]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [46]:
%pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [47]:
%pip install matplotlib

In [48]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [49]:
import networkx as nx

In [50]:
import matplotlib.pyplot as plt

In [51]:
import psycopg2

In [52]:
import pandas as pd

In [53]:
DB_NAME = 'mapData'
DB_USER = 'postgres'
DB_PASSWORD = '1234'
DB_HOST = 'localhost'

In [76]:
# Establish a connection to the database
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)

In [77]:
# Create a cursor object to execute SQL queries
cur = conn.cursor()

In [78]:
# Fetch data from table
cur.execute('SELECT * FROM edges_lrt_mrt')
edges = cur.fetchall()

cur.execute('SELECT id, latitude, longitude FROM merged_mrt_lrt')
positions = cur.fetchall()

In [57]:
# print the position data
print(positions)

[(44, 3.01753842, 101.6129545), (136, 3.142421601, 101.7202117), (33, 3.076154052, 101.7113372), (34, 3.064227861, 101.7080286), (35, 3.059785605, 101.6920342), (36, 3.061413714, 101.6871381), (37, 3.062272, 101.670654), (38, 3.0620987, 101.6627423), (39, 3.054841361, 101.656468), (40, 3.050638642, 101.6442686), (41, 3.048311297, 101.6209228), (42, 3.033643559, 101.6161685), (43, 3.022975961, 101.6133461), (45, 3.007942706, 101.6056225), (46, 3.000027223, 101.5966946), (47, 2.996097274, 101.575498), (48, 3.231399278, 101.7242403), (49, 3.21980733, 101.7218576), (54, 3.165292218, 101.7318589), (55, 3.164560931, 101.7244835), (56, 3.160024057, 101.7190732), (57, 3.159173394, 101.714153), (58, 3.161631031, 101.7066476), (59, 3.15648462, 101.7022776), (61, 3.142492499, 101.6953129), (63, 3.127677475, 101.6789561), (64, 3.118943189, 101.6732605), (65, 3.115718903, 101.6686076), (51, 3.199237959, 101.73751), (50, 3.205911293, 101.7318969), (20, 3.178490429, 101.6956353), (21, 3.173485586, 10

In [79]:
# Convert the fetched data into a pandas DataFrame for easier manipulation
columns = ['id','source_station_id', 'destination_station_id', 'distance']
df = pd.DataFrame(edges, columns=columns)

In [59]:
cur.close()
conn.close()

In [207]:
import sqlite3
import time
import heapq
import math
import networkx as nx

In [209]:
def fetch_all_coordinates_from_db(db_path):
    """
    Fetch all coordinates (latitude and longitude) for the nodes from the database.

    Parameters:
    db_path (str): Path to the SQLite database file.

    Returns:
    list: A list of tuples containing (node_id, latitude, longitude).
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    query = 'SELECT id, latitude, longitude FROM merged_mrt_lrt'
    cursor.execute(query)
    positions = cursor.fetchall()
    conn.close()
    return positions

In [86]:
print(edges)

[(309, 1, 2, 3), (310, 2, 3, 2), (311, 3, 4, 3), (312, 4, 5, 3), (313, 5, 6, 3), (314, 6, 7, 3), (315, 7, 8, 3), (316, 8, 9, 3), (317, 9, 10, 3), (318, 10, 11, 3), (319, 11, 12, 3), (320, 12, 13, 3), (321, 13, 14, 4), (322, 14, 15, 3), (323, 15, 16, 3), (324, 16, 17, 3), (325, 17, 18, 5), (326, 19, 20, 3), (327, 20, 21, 2), (328, 21, 22, 3), (329, 22, 23, 3), (330, 23, 24, 3), (331, 24, 25, 3), (332, 25, 26, 3), (333, 26, 27, 3), (334, 27, 28, 3), (335, 28, 29, 3), (336, 29, 30, 4), (337, 30, 31, 4), (338, 31, 32, 4), (339, 32, 33, 4), (340, 33, 34, 4), (341, 34, 35, 5), (342, 35, 36, 3), (343, 36, 37, 4), (344, 37, 38, 3), (345, 38, 39, 4), (346, 39, 40, 4), (347, 40, 41, 5), (348, 41, 42, 4), (349, 42, 43, 4), (350, 43, 44, 3), (351, 44, 45, 4), (352, 45, 46, 4), (353, 46, 47, 6), (354, 48, 49, 4), (355, 49, 50, 4), (356, 50, 51, 3), (357, 51, 52, 5), (358, 52, 53, 3), (359, 53, 54, 2), (360, 54, 55, 3), (361, 55, 56, 3), (362, 56, 57, 3), (363, 57, 58, 3), (364, 58, 59, 3), (365, 59

In [210]:
# Create a graph and add nodes with coordinates
G = nx.Graph()
for node_id, lat, long in positions:
    G.add_node(node_id, pos=(lat, long))

# Add edges to the graph
for edge_id, source, destination, distance in edges:
    G.add_edge(source, destination, weight=distance)


In [211]:
def heuristic_distance(station1, station2):
    """
    Calculate the Euclidean distance between two stations.

    Parameters:
    station1 (tuple): The (latitude, longitude) coordinates of the first station.
    station2 (tuple): The (latitude, longitude) coordinates of the second station.

    Returns:
    float: The Euclidean distance between the two stations.
    """
    x1, y1 = station1
    x2, y2 = station2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [212]:
import psutil
import os

def get_memory_usage():
    """
    Get the current memory usage of the process.

    Returns:
    int: Memory usage in bytes.
    """
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

In [247]:
def astar_shortest_path(graph, start_station_id, end_station_id):
    """
    Calculate the shortest path between two stations using the A* algorithm.

    Parameters:
    graph (networkx.Graph): The graph representing the station network.
    start_station_id (int): The ID of the start station.
    end_station_id (int): The ID of the end station.

    Returns:
    tuple: The shortest path from the start station to the end station, runtime in milliseconds, number of steps taken, and memory usage in MB.
    """
    # Record the start time
    start_time = time.time()

    # Get initial memory usage
    initial_memory = get_memory_usage()

    # Define the priority queue for the open set
    open_set = [(0, start_station_id)]
    # Define the set of visited stations
    visited = set()
    # Define the dictionary to store the actual distance from the start station to each station
    g_score = {start_station_id: 0}
    # Define the dictionary to store the estimated total distance from the start station to each station
    f_score = {start_station_id: heuristic_distance(graph.nodes[start_station_id]['pos'], graph.nodes[end_station_id]['pos'])}
    # Define the dictionary to store the previous station in the shortest path
    came_from = {}

    # Initialize the number of steps taken
    steps = 0

    while open_set:
        # Increment the number of steps
        steps += 1
        
        # Get the station with the lowest f_score
        _, current_station = heapq.heappop(open_set)
        
        if current_station == end_station_id:
            # Record the end time
            end_time = time.time()
            # Calculate the runtime in milliseconds
            runtime = (end_time - start_time) * 1000

            # Get final memory usage
            final_memory = get_memory_usage()
            memory_usage = (final_memory - initial_memory) / 1024 / 1024  # Convert to MB
            
            # Reconstruct the path from the end station to the start station
            path = [current_station]
            while current_station != start_station_id:
                current_station = came_from[current_station]
                path.append(current_station)
            path.reverse()
            return path, runtime, steps, memory_usage
        
        # Add the current station to the visited set
        visited.add(current_station)
        
        # Explore the neighbors of the current station
        for neighbor in graph.neighbors(current_station):
            if neighbor in visited:
                continue
            
            # Calculate the tentative g_score for the neighbor
            tentative_g_score = g_score[current_station] + graph[current_station][neighbor]['weight']
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                # Update the g_score and f_score for the neighbor
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic_distance(graph.nodes[neighbor]['pos'], graph.nodes[end_station_id]['pos'])
                # Update the came_from dictionary
                came_from[neighbor] = current_station
                # Add the neighbor to the open set
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    # If no path is found, return None
    return None, None, None, None

In [248]:
# Define start and end station IDs
start_station_id = 150
end_station_id = 48

# Calculate the shortest path using A* algorithm
path, runtime, steps, memory_usage = astar_shortest_path(G, start_station_id, end_station_id)

# Count the number of stations in the path
count = len(path)

# Print the results
print("Shortest path:", path)
print("Number of stations in path:", count)
print("Runtime:", runtime, "milliseconds")
print("Number of steps:", steps)
print("Memory usage:", memory_usage, "MB")

Shortest path: [150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 106, 105, 104, 103, 56, 55, 54, 53, 52, 51, 50, 49, 48]
Number of stations in path: 28
Runtime: 1.5730857849121094 milliseconds
Number of steps: 123
Memory usage: 0.0 MB


In [232]:
def dijkstra_shortest_path(graph, start_station_id, end_station_id):
    """
    Calculate the shortest path between two stations using Dijkstra's algorithm.

    Parameters:
    graph (networkx.Graph): The graph representing the station network.
    start_station_id (int): The ID of the start station.
    end_station_id (int): The ID of the end station.

    Returns:
    tuple: The shortest path from the start station to the end station, runtime in milliseconds, number of steps taken, and memory usage in MB.
    """
    # Record the start time
    start_time = time.time()

    # Get initial memory usage
    initial_memory = get_memory_usage()

    # Define the priority queue for the open set
    open_set = [(0, start_station_id)]
    # Define the dictionary to store the actual distance from the start station to each station
    distances = {start_station_id: 0}
    # Define the dictionary to store the previous station in the shortest path
    came_from = {}

    # Initialize the number of steps taken
    steps = 0

    while open_set:
        # Increment the number of steps
        steps += 1
        
        # Get the station with the smallest distance
        current_distance, current_station = heapq.heappop(open_set)
        
        if current_station == end_station_id:
            # Record the end time
            end_time = time.time()
            # Calculate the runtime in milliseconds
            runtime = (end_time - start_time) * 1000

            # Get final memory usage
            final_memory = get_memory_usage()
            memory_usage = (final_memory - initial_memory) / 1024 / 1024  # Convert to MB
            
            # Reconstruct the path from the end station to the start station
            path = [current_station]
            while current_station != start_station_id:
                current_station = came_from[current_station]
                path.append(current_station)
            path.reverse()
            return path, runtime, steps, memory_usage
        
        # Explore the neighbors of the current station
        for neighbor in graph.neighbors(current_station):
            distance = current_distance + graph[current_station][neighbor]['weight']
            
            if neighbor not in distances or distance < distances[neighbor]:
                # Update the distance for the neighbor
                distances[neighbor] = distance
                # Update the came_from dictionary
                came_from[neighbor] = current_station
                # Add the neighbor to the open set
                heapq.heappush(open_set, (distance, neighbor))
    
    # If no path is found, return None
    return None, None, None, None

In [233]:
# Define start and end station IDs
start_station_id = 150
end_station_id = 48

# Calculate the shortest path using Dijkstra's algorithm
path, runtime, steps, memory_usage = dijkstra_shortest_path(G, start_station_id, end_station_id)

# Count number of station in path array
count = len(path)

# Print the results
print("Shortest path:", path)
print("Number of stations in path:", count)
print("Runtime:", runtime, "milliseconds")
print("Number of steps:", steps)
print("Memory usage:", memory_usage, "MB")

Shortest path: [150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 106, 105, 104, 103, 56, 55, 54, 53, 52, 51, 50, 49, 48]
Number of stations in path: 28
Runtime: 0.06270408630371094 milliseconds
Number of steps: 125
Memory usage: 0.0 MB


In [234]:
import random

def ant_colony_optimization(graph, start_station_id, end_station_id, num_ants=10, num_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, pheromone_deposit=1.0):
    """
    Calculate the shortest path between two stations using Ant Colony Optimization (ACO).

    Parameters:
    graph (networkx.Graph): The graph representing the station network.
    start_station_id (int): The ID of the start station.
    end_station_id (int): The ID of the end station.
    num_ants (int): Number of ants to use in the algorithm.
    num_iterations (int): Number of iterations to run the algorithm.
    alpha (float): Parameter to control the influence of pheromone.
    beta (float): Parameter to control the influence of distance.
    evaporation_rate (float): Rate at which the pheromone evaporates.
    pheromone_deposit (float): Amount of pheromone deposited by each ant.

    Returns:
    tuple: The shortest path from the start station to the end station, runtime in milliseconds, number of steps taken, and memory usage in MB.
    """
    # Record the start time
    start_time = time.time()

    # Get initial memory usage
    initial_memory = get_memory_usage()

    # Initialize pheromone levels on each edge
    pheromone = {}
    for edge in graph.edges:
        pheromone[edge] = 1.0
        pheromone[(edge[1], edge[0])] = 1.0  # Ensure bidirectional pheromone initialization

    def choose_next_station(current_station, visited):
        probabilities = []
        neighbors = list(graph.neighbors(current_station))
        for neighbor in neighbors:
            if neighbor not in visited:
                edge = (current_station, neighbor)
                tau = pheromone[edge] ** alpha
                weight = graph[current_station][neighbor]['weight']
                if weight > 0:  # Check to avoid division by zero
                    eta = (1.0 / weight) ** beta
                    probabilities.append((neighbor, tau * eta))
        if not probabilities:
            return None
        total = sum(prob for _, prob in probabilities)
        r = random.uniform(0, total)
        cumulative = 0.0
        for neighbor, prob in probabilities:
            cumulative += prob
            if cumulative >= r:
                return neighbor
        return None

    def update_pheromone(paths, best_path):
        # Evaporate pheromone
        for edge in pheromone:
            pheromone[edge] *= (1.0 - evaporation_rate)
        # Deposit new pheromone
        for path in paths:
            for i in range(len(path) - 1):
                edge = (path[i], path[i + 1])
                pheromone[edge] += pheromone_deposit / graph[path[i]][path[i + 1]]['weight']
                pheromone[(edge[1], edge[0])] += pheromone_deposit / graph[path[i]][path[i + 1]]['weight']  # Bidirectional update
        # Deposit additional pheromone for the best path
        for i in range(len(best_path) - 1):
            edge = (best_path[i], best_path[i + 1])
            pheromone[edge] += pheromone_deposit / graph[best_path[i]][best_path[i + 1]]['weight']
            pheromone[(edge[1], edge[0])] += pheromone_deposit / graph[best_path[i]][best_path[i + 1]]['weight']  # Bidirectional update

    best_path = None
    best_path_length = float('inf')

    for _ in range(num_iterations):
        paths = []
        for _ in range(num_ants):
            current_station = start_station_id
            path = [current_station]
            visited = set(path)
            while current_station != end_station_id:
                next_station = choose_next_station(current_station, visited)
                if next_station is None:
                    break
                path.append(next_station)
                visited.add(next_station)
                current_station = next_station
            if path[-1] == end_station_id:
                paths.append(path)
                path_length = sum(graph[path[i]][path[i + 1]]['weight'] for i in range(len(path) - 1))
                if path_length < best_path_length:
                    best_path = path
                    best_path_length = path_length
        update_pheromone(paths, best_path)

    # Record the end time
    end_time = time.time()
    # Calculate the runtime in milliseconds
    runtime = (end_time - start_time) * 1000
    # Get final memory usage
    final_memory = get_memory_usage()
    memory_usage = (final_memory - initial_memory) / 1024 / 1024  # Convert to MB
    # Number of steps is the number of iterations times the number of ants
    steps = num_iterations * num_ants

    return best_path, runtime, steps, memory_usage

In [237]:
# Define start and end station IDs
start_station_id = 150
end_station_id = 48

# Calculate the shortest path using Ant Colony Optimization (ACO)
path, runtime, steps, memory_usage = ant_colony_optimization(G, start_station_id, end_station_id)

# Count number of station in path array
count = len(path)

# Print the results
print("Shortest path:", path)
print("Number of stations in path:", count)
print("Runtime:", runtime, "milliseconds")
print("Number of steps:", steps)
print("Memory usage:", memory_usage, "MB")

Shortest path: [150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 157, 158, 159, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48]
Number of stations in path: 31
Runtime: 192.9161548614502 milliseconds
Number of steps: 1000
Memory usage: 0.0 MB
